In [74]:
import sqlite3
import pandas as pd
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import numpy as np
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [75]:
desktop_path = os.path.expanduser("~/Desktop/hedging-against-turkish-inflation/notebooks/database_final.db")

if not os.path.isfile(desktop_path):
    raise FileNotFoundError(f"Database file not found at {desktop_path}")

conn = sqlite3.connect(desktop_path)

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:")
print(tables)

Tables in the database:
      name
0  STOCKEX
1      BTC
2     GOLD
3     BOND
4      HPI
5      INF


In [76]:
df_stock = pd.read_sql_query("SELECT * from STOCKEX", conn)
df_gold = pd.read_sql_query("SELECT * from GOLD", conn)
df_bond = pd.read_sql_query("SELECT * from BOND", conn)
df_hpi = pd.read_sql_query("SELECT * from HPI", conn)
df_btc = pd.read_sql_query("SELECT * from BTC", conn)

# Markowitz Portfolio:

In [77]:
stock_ret = df_stock["real_return_stockex"]
gold_ret = df_gold["real_return_gold"]
bond_ret = df_bond["real_return_bond"]
hpi_ret = df_hpi["real_return_hpi"]
btc_ret = df_btc["real_return_bitcoin"]

In [83]:
returns = pd.concat([stock_ret, gold_ret, bond_ret, hpi_ret, btc_ret], axis=1)
returns.columns = ['Stock', 'Gold', 'Bond', 'HPI', 'Bitcoin']

mean_returns = returns.mean()
cov_matrix = returns.cov()

num_assets = len(mean_returns)

def portfolio_performance(weights, mean_returns, cov_matrix):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_volatility

def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0.2273):
    p_return, p_volatility = portfolio_performance(weights, mean_returns, cov_matrix)
    sharpe_ratio = (p_return - risk_free_rate) / p_volatility
    return -sharpe_ratio  

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) 
bounds = tuple((0, 1) for _ in range(num_assets))  

initial_guess = num_assets * [1. / num_assets, ]

result = minimize(negative_sharpe_ratio, initial_guess,
                  args=(mean_returns, cov_matrix),
                  method='SLSQP', bounds=bounds, constraints=constraints)

optimal_weights = result.x

optimal_return, optimal_volatility = portfolio_performance(optimal_weights, mean_returns, cov_matrix)

weights_percentages = optimal_weights * 100

print("Optimal Portfolio Weights (%):")
for asset, weight in zip(returns.columns, weights_percentages):
    print(f"{asset}: {weight:.2f}%")

print(f"\nExpected Portfolio Return: {optimal_return:.2%}")
print(f"Expected Portfolio Volatility: {optimal_volatility:.2%}")

Optimal Portfolio Weights (%):
Stock: 12.16%
Gold: 0.00%
Bond: 0.00%
HPI: 86.68%
Bitcoin: 1.16%

Expected Portfolio Return: 81.88%
Expected Portfolio Volatility: 90.39%
